In [1]:
from fabric import ThreadingGroup, Connection
from chi import server, ssh
import os
import warnings
import time
import sys

sys.path.append("../../scripts")
from utilities import *

warnings.filterwarnings("ignore")


> CONFIG

In [2]:
sshUser = "ubuntu"
server_ips = ["64.131.114.157"]
server_local_ips = {}
client_ips = ["64.131.114.158"]

storageType = {"HDD": "/HDD", "NVME": "/NVME_RAID-0"}
SERVER_WORKING_DIR = storageType["NVME"]


In [3]:
# These groups are for processing in parallel
serverConnections = [Connection(i, user=sshUser) for i in server_ips]
clientConnections = [Connection(i, user=sshUser) for i in client_ips]
allConnections = serverConnections + clientConnections
serverGroup = ThreadingGroup.from_connections(serverConnections)
clientGroup = ThreadingGroup.from_connections(clientConnections)
allGroup = ThreadingGroup.from_connections(allConnections)

# Get local IP for server
local_ips = serverGroup.run("hostname -I | cut -d ' ' -f 1", hide=True)
for i in local_ips:
    server_local_ips[i.host] = local_ips[i].stdout.strip()


In [4]:
targetThreads = [1]
numClients = [i * len(client_ips) for i in targetThreads]


> INSTALLATION

In [5]:
time.sleep(2)

for ip in server_ips + client_ips:
    server.wait_for_tcp(ip, 22, timeout=1200)


In [ ]:
serverGroup.put("../../scripts/TimescaleDB/initTimescaleDBServer.sh")
serverGroup.run(f"sudo chmod -R 777 {SERVER_WORKING_DIR}; bash initTimescaleDBServer.sh {SERVER_WORKING_DIR} true")
serverGroup.close()


In [ ]:
clientGroup.put("../../scripts/initXBenchClient.sh")
clientGroup.run("bash initXBenchClient.sh")
clientGroup.close()


In [ ]:
runWorkload(
    "INSERT_UNARY_SEQ",
    targetThreads,
    server_local_ips[server_ips[0]],
    9493,
    "TimescaleDB",
    client_ips,
    10,
    0,
    serverGroup,
    SERVER_WORKING_DIR,
    "ubuntu",
)
